In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = "C:\\evdoxiataka\\effi_user_study1\\code\\"
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder= "C:\\evdoxiataka\\effi_user_study1\\"

from utils.data_loading import get_all_participant_feedback_clean

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Data Loading

In [2]:
## LOAD FEEDBACK
prolific_export_filePath = folder+"data\\collected_data\\prolific_export_demographics.csv"
interaction_logs_filePath = folder+"data\\collected_data\\LOGS\\"
feedback_df = get_all_participant_feedback_clean(prolific_export_filePath, interaction_logs_filePath)
# feedback_df.to_csv("Feedback_final.csv")

In [3]:
## LOAD APPLICATIONS SHOWN TO PARTICIPANTS
applications_df = pd.read_csv(folder+"data\\processed_data\\Applications_Converted.csv")
ft_weight_dict = {}

for i in range(0,49):
    ft_weight_dict.update({applications_df["ft_name_"+ str(i)][0]:applications_df["ft_weight_"+ str(i)][0]})

## Average number of clicks that a user did on a function with std

In [ ]:
function_group = feedback_df.groupby(by = ["ID","Function"])

function_count = function_group.size().unstack(fill_value=0).stack()

parti_list = []
function_list = []
function_count_list = function_count.tolist()

# print(len(function_count_list))

for i in range(0,len(function_count.index)):
    parti_list.append(function_count.index[i][0])
    function_list.append(function_count.index[i][1])

parti_func_df = pd.DataFrame()

parti_func_df["Participant"] = parti_list
parti_func_df["Function"] = function_list
parti_func_df["Participant_Function_Count"] = function_count_list
parti_func_df

In [ ]:
data = []

for func in parti_func_df["Function"].unique():
    
    data.append(
    {
        "Function" : func,
        "Total" : parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.sum(),
        "Mean": round(parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.mean(),2),
        "Std": round(parti_func_df[parti_func_df["Function"]== func ].Participant_Function_Count.std(),2),
        "Number of Participants Clicked" : parti_func_df[(parti_func_df["Function"]== func) & (parti_func_df["Participant_Function_Count"] != 0)].shape[0]
    }
    )
    
each_function_df = pd.DataFrame(data)
each_function_df.sort_values(by='Total', ascending=False, inplace = True)
each_function_df.reset_index(drop = True, inplace = True)
each_function_df

each_function_df.replace({'Function' : { "select_applications" :'Select an application in APPs table', 
                                        "apply_refine_search":"Filter for an attribute in APPs table",
                                        "apply_similar_application_range": "Pushing Apply button (in Compare)",
                                        "similar_application": "Selecting an Application in Scatter plot (in Compare)",
                                        "Similar_Application_table_Similarity_of_each_attribute": "Sorting the table on Similarity of each attribute  (in Compare)",
                                        "select_reject_application":'Give rating and weight for an Application',
                                        "Attribute_Info_Importance":"Sorting Attribute information  Importance",
                                        "Applications_Prediction_Confidence":"Sorting Applications Prediction Confidence",
                                        "Applications_Fairness_raiting":"Sorting Applications Fairness Rating",
                                        "clear_refine_search":"Pushing Clear button (in Applications filter)",
                                        "Application_table_Weight":"Sorting Application table on Weight",
                                        "select_suggest_change_button":"Pushing Suggest changes button",
                                        "save_suggest_change":"Pushing Suggest change Save button",
                                        "suggest_change":"Changed values in Suggest changes",
                                        "Applications_ID_Name":"Sorting Applications ID Number",
                                        "Applications_Predicted_decision":"Sorting Applications Predicted decision",
                                        "click_show_more":"Pushing the link of show_more",
                                        "click_show_less":"Pushing the link of show_less",
                                        "unselect_applications":"Unselect Applications column",
                                        "select_human_offer":"Selecting Application involve a human officer (offer is a typo of officer)",
                                        "display_attribute_info":"Displaying Applications Attribute Info",
                                        "hidden_attribute_info":"Hiding Applications Attribute Info",
                                        "clear_suggest_change":"Pushing Suggest change Clear button",
                                        "clear_similar_application_range":"Pushing Clear button (in Compare)",
                                       "Click on Feature Combination Button":"Click on Feature Combination Button",
                                       "Clicked-on-Causal-Graph-Node":"Click on node of causal graph"}},inplace=True)
each_function_df

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])


bars = ax.bar(each_function_df["Function"],each_function_df["Total"], width=0.5, color = "green")
ax.bar_label(bars)
plt.title("Clicks vs Function")
plt.xlabel('Function')
plt.ylabel('Number of Clicks')
fig.autofmt_xdate()
plt.show()

## How many times each application was viewed w.r.t each user

In [ ]:
feedback_df[feedback_df["Function"] == "select_applications"]

display(feedback_df[feedback_df["Function"] == "select_applications"].head())
print(feedback_df[feedback_df["Function"] == "select_applications"]["ID"].nunique())

In [ ]:
app_group = feedback_df[feedback_df["Function"] == "select_applications"].groupby(by = ["ID","App ID"])

app_count = app_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(app_count.index)):    
    data.append(
    {
        "Participant" : app_count.index[i][0],
        "App ID" : app_count.index[i][1],
        "Count": app_count.values[i]
    }
    )
    
parti_app_df = pd.DataFrame(data)
parti_app_df

In [ ]:
data = []

for app_id in parti_app_df["App ID"].unique():    
    data.append(
    {
        "Application" : app_id,
        "Total" : parti_app_df[parti_app_df["App ID"]== app_id ].Count.sum(),
        "Mean": round(parti_app_df[parti_app_df["App ID"]== app_id ].Count.mean(),2),
        "Std": round(parti_app_df[parti_app_df["App ID"]== app_id ].Count.std(),2),
        "Number of Participants Viewed" : parti_app_df[(parti_app_df["App ID"]== app_id) & (parti_app_df["Count"] != 0)].shape[0]
    }
    )
    
each_app_df = pd.DataFrame(data)
each_app_df

each_app_df.sort_values(by='Total', ascending=False, inplace = True)
each_app_df.fillna("NA", inplace = True)
each_app_df.reset_index(drop = True, inplace = True)
each_app_df

# each_app_df.to_csv("How many times each application was viewed w.r.t each user.csv")

## Which attribute was most filtered for w.r.t each participant

In [ ]:
feedback_df[feedback_df["Function"] == "apply_refine_search"]

display(feedback_df[feedback_df["Function"] == "apply_refine_search"].head())
print("No.Part.:",feedback_df[feedback_df["Function"] == "apply_refine_search"]["ID"].nunique())

In [ ]:
attribute_group = feedback_df[feedback_df["Function"] == "apply_refine_search"].groupby(by = ["ID","Attribute"])

attribute_count = attribute_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(attribute_count.index)):
    
    if attribute_count.index[i][1] in list(ft_weight_dict.keys()):
        data.append(
        {
            "Participant" : attribute_count.index[i][0],
            "Attribute" : attribute_count.index[i][1],
            "Count": attribute_count.values[i]
        }
        )
    
parti_attri_df = pd.DataFrame(data)
parti_attri_df

In [ ]:
data = []

for attribute in list(ft_weight_dict.keys()):
    
    data.append(
    {
        "Attribute" : attribute,
        "Total" : parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.sum(),
        "Mean": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.mean(),2),
        "Std": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.std(),2),
        "Number of Participants Filtered" : parti_attri_df[(parti_attri_df["Attribute"]== attribute) & (parti_attri_df["Count"] != 0)].shape[0]
    }
    )
    
each_attri_df = pd.DataFrame(data)
each_attri_df

each_attri_df.sort_values(by='Total', ascending=False, inplace = True)
each_attri_df.fillna("NA", inplace = True)
each_attri_df.reset_index(drop = True, inplace = True)
each_attri_df
# each_attri_df.to_csv("Attribute was most filtered for w.r.t each participant.csv")

## Applications_Prediction_Confidence : Ascended or Descended w.r.t each User

In [ ]:
# feedback_df[feedback_df["Function"] == "Applications_Prediction_Confidence"]

In [ ]:
pc_group= feedback_df[feedback_df["Function"] == "Applications_Prediction_Confidence"].groupby(by = ["ID","Value"])

pc_count = pc_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(pc_count.index)):
    
    data.append(
    {
        "Participant" : pc_count.index[i][0],
        "PC_Value" : pc_count.index[i][1],
        "Count": pc_count.values[i]
    }
    )
    
parti_pc_df = pd.DataFrame(data)
parti_pc_df

In [ ]:
data = []

for pc_action in feedback_df[feedback_df["Function"] == "Applications_Prediction_Confidence"]["Value"].unique():
    
    data.append(
    {
        "PC_Value" : pc_action,
        "Total" : parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.sum(),
        "Mean": round(parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.mean(),2),
        "Std": round(parti_pc_df[parti_pc_df["PC_Value"]== pc_action ].Count.std(),2),
        "Number of Participants Clicked" : parti_pc_df[(parti_pc_df["PC_Value"]== pc_action) & (parti_pc_df["Count"] != 0)].shape[0]
    }
    )
    
each_pc_df = pd.DataFrame(data)

each_pc_df.sort_values(by='Total', ascending=False, inplace = True)
each_pc_df.fillna("NA", inplace = True)
each_pc_df.reset_index(drop = True, inplace = True)
display(each_pc_df)

## How many participants selected to view a similar application?

In [ ]:
p_ids_similar = feedback_df[feedback_df["Function"] == "similar_application"]['ID'].unique()

In [ ]:
len(p_ids_similar)

## How many participants clicked on Feature Combinations buttons

In [ ]:
feedback_df[feedback_df["Function"] == "Click on Feature Combination Button"]['ID'].nunique()


## Which attribute was most clicked on the causal graph w.r.t each participant

In [ ]:
feedback_df[feedback_df["Function"] == "Clicked-on-Causal-Graph-Node"]

display(feedback_df[feedback_df["Function"] == "Clicked-on-Causal-Graph-Node"].head())
print("No. Part.:",feedback_df[feedback_df["Function"] == "Clicked-on-Causal-Graph-Node"]["ID"].nunique())

In [ ]:
attribute_group = feedback_df[feedback_df["Function"] == "Clicked-on-Causal-Graph-Node"].groupby(by = ["ID","Value"])

attribute_count = attribute_group.size().unstack(fill_value=0).stack()

data = []

for i in range(0,len(attribute_count.index)):
    
    if attribute_count.index[i][1] in list(ft_weight_dict.keys()):
        data.append(
        {
            "Participant" : attribute_count.index[i][0],
            "Attribute" : attribute_count.index[i][1],
            "Count": attribute_count.values[i]
        }
        )
    
parti_attri_df = pd.DataFrame(data)
parti_attri_df

In [ ]:
data = []

for attribute in list(ft_weight_dict.keys()):
    
    data.append(
    {
        "Attribute" : attribute,
        "Total" : parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.sum(),
        "Mean": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.mean(),2),
        "Std": round(parti_attri_df[parti_attri_df["Attribute"]== attribute ].Count.std(),2),
        "Number of Participants Clicked Node" : parti_attri_df[(parti_attri_df["Attribute"]== attribute) & (parti_attri_df["Count"] != 0)].shape[0]
    }
    )
    
each_attri_df = pd.DataFrame(data)
each_attri_df

each_attri_df.sort_values(by='Total', ascending=False, inplace = True)
each_attri_df.fillna("NA", inplace = True)
each_attri_df.reset_index(drop = True, inplace = True)
each_attri_df
# each_attri_df.to_csv("Attribute was most clicked on causal graph w.r.t each participant.csv")

## How many applications were rated as fair/unfair w.r.t each participant

In [ ]:
rating_group= feedback_df[feedback_df["Function"] == "select_reject_application"].groupby(by = ["ID","Attribute"])

rating_count = rating_group.size().unstack(fill_value=0).stack()


data = []

for i in range(0,len(rating_count.index)):
    
    data.append(
    {
        "Participant" : rating_count.index[i][0],
        "Rating" : rating_count.index[i][1],
        "Count": rating_count.values[i]
    }
    )
    
parti_rating_df = pd.DataFrame(data)
parti_rating_df

In [ ]:
data = []

for rating in parti_rating_df["Rating"].unique():
    
    data.append(
    {
        "Rating" : rating,
        "Total" : parti_rating_df[parti_rating_df["Rating"]== rating ].Count.sum(),
        "Mean": round(parti_rating_df[parti_rating_df["Rating"]== rating ].Count.mean(),2),
        "Std": round(parti_rating_df[parti_rating_df["Rating"]== rating ].Count.std(),2),
        "Number of Participants Clicked" : parti_rating_df[(parti_rating_df["Rating"]== rating) & (parti_rating_df["Count"] != 0)].shape[0]

    }
    )
    
each_rating_df = pd.DataFrame(data)

each_rating_df.sort_values(by='Total', ascending=False, inplace = True)
each_rating_df.fillna("NA", inplace = True)
each_rating_df.reset_index(drop = True, inplace = True)
each_rating_df
# each_rating_df.to_csv("select_reject_application.csv")

## How often participants gave unfair labels w.r.t each participant

In [ ]:
parti_rating_df2 = parti_rating_df[parti_rating_df['Rating']=='unfair']
data = []

for participant in parti_rating_df2["Participant"].unique():
    
    data.append(
    {
        "Participant" : participant,
        "Total_unfair_Ratings" : parti_rating_df2[parti_rating_df2["Participant"]== participant].Count.sum(),
        # "Mean": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.mean(),2),
        # "Std": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.std(),2)
    }
    )    
    
each_participant_rating_df = pd.DataFrame(data)
each_participant_rating_df.sort_values(by='Total_unfair_Ratings', ascending=False, inplace = True)
each_participant_rating_df.fillna("NA", inplace = True)
each_participant_rating_df.reset_index(drop = True, inplace = True)
# each_participant_rating_df.to_csv("How often participants gave unfair labels w.r.t each participant.csv")

## How often participants suggested weight changes w.r.t each participant

In [6]:
feedback_group =feedback_df[(feedback_df["Function"] == "select_reject_application")]

In [ ]:
feedback_group

In [7]:
labels_with_weights_all = feedback_group[[True if isinstance(val,str) and val!="[]" else False for val in feedback_group["Value"]]]
labels_with_weights_unfair =labels_with_weights_all[labels_with_weights_all['Attribute']=='unfair']
labels_with_weights_ids= labels_with_weights_unfair['ID']

labels_without_weights_all = feedback_group[[True if isinstance(val,str) and val=="[]" else False for val in feedback_group["Value"]]]
labels_without_weights_unfair = labels_without_weights_all[labels_without_weights_all['Attribute']=='unfair']
labels_without_weights_ids = labels_without_weights_all['ID']


In [ ]:
participants_weight_group= feedback_group.groupby(by = ["ID","Value"])

participant_count = participants_weight_group.size().unstack(fill_value=0).stack()

data = []
for i in range(0,len(participant_count.index)):    
    # if participant_count.index[i][1] in feedback_df[(feedback_df["Function"] == "OKBUTTON_CLICKED_DECIDE_MODAL_Applications_List") & (feedback_df["Value"] != "[]")]["Value"].unique():
    if participant_count.index[i][1] in labels_with_weights_all["Value"].unique():    
        data.append(
        {
            "Participant" : participant_count.index[i][0],
            "Weight" : participant_count.index[i][1],
            "Count": participant_count.values[i]
        }
        )
    
participant_weight_df = pd.DataFrame(data)
participant_weight_df

In [ ]:
data = []

for participant in participant_weight_df["Participant"].unique():
    
    data.append(
    {
        "Participant" : participant,
        "No. Weight Changes" : participant_weight_df[participant_weight_df["Participant"]== participant ].Count.sum(),
    }
    )
    
    
each_participant_weight_df = pd.DataFrame(data)

each_participant_weight_df.sort_values(by='No. Weight Changes', ascending=False, inplace = True)
each_participant_weight_df.fillna("NA", inplace = True)
each_participant_weight_df.reset_index(drop = True, inplace = True)
each_participant_weight_df
# each_participant_weight_df.to_csv("How often participants suggested weight changes w.r.t each participant.csv")

# Can't delete these entries because the participant is labeling different applications in these entries. 

## How often participants gave unfair labels w.r.t each participant

In [ ]:
rating_group= feedback_df[feedback_df["Function"] == "select_reject_application"].groupby(by = ["ID","Attribute"])

rating_count = rating_group.size().unstack(fill_value=0).stack()


data = []

for i in range(0,len(rating_count.index)):
    
    data.append(
    {
        "Participant" : rating_count.index[i][0],
        "Rating" : rating_count.index[i][1],
        "Count": rating_count.values[i]
    }
    )
    
parti_rating_df = pd.DataFrame(data)
parti_rating_df

In [ ]:
parti_rating_df2 = parti_rating_df[parti_rating_df['Rating']=='unfair']
data = []

for participant in parti_rating_df2["Participant"].unique():
    
    data.append(
    {
        "Participant" : participant,
        "Total_unfair_Ratings" : parti_rating_df2[parti_rating_df2["Participant"]== participant].Count.sum(),
        # "Mean": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.mean(),2),
        # "Std": round(parti_rating_df[parti_rating_df["Participant"]== participant].Count.std(),2)
    }
    )    
    
each_participant_rating_df = pd.DataFrame(data)
each_participant_rating_df.sort_values(by='Total_unfair_Ratings', ascending=False, inplace = True)
each_participant_rating_df.fillna("NA", inplace = True)
each_participant_rating_df.reset_index(drop = True, inplace = True)
each_participant_rating_df
# each_participant_rating_df.to_csv("How often participants gave unfair labels w.r.t each participant.csv")

## Check Feedback

### How many participants provided feedback for the same application more than once

In [42]:
import collections
import ast
feedback_group = feedback_df[feedback_df["Function"] == "select_reject_application"]
for idx in feedback_group.index:
    feedback_group.at[idx, 'Value'] = ast.literal_eval(feedback_group.loc[idx]['Value'])

## checks
participants_checked_app_multiple_times=[]
for part in feedback_group['ID'].unique():
    part_feedback = feedback_group[feedback_group["ID"]== part]
    if len(part_feedback)> len(part_feedback['App ID'].unique()):
        # print(part_feedback['App ID'])
        # print([item for item, count in collections.Counter(part_feedback['App ID'].tolist()).items() if count > 1])
        participants_checked_app_multiple_times.append(part)
len(participants_checked_app_multiple_times)

35

In [43]:
def update_clear_value(cl_value, el):
    """
        cl_value: List of elements {'attribute':<>,'value':<>}
        el: Dict element {'attribute':<>,'value':<>}
    """
    ## If attribute already in cl_value
    ## update value
    ## else add element to cl_value
    if len(cl_value):                    
        attr_found = False
        for el_prev_i,el_prev in enumerate(cl_value):
            if el_prev['attribute'] == el['attribute']:
                cl_value[el_prev_i]['value'] = el['value']
                attr_found = True
                break
        if not attr_found:
            cl_value.append(el)
    else:
        cl_value.append(el)
    return cl_value


import copy
from collections import Counter
count_idx = -1
df_all = pd.DataFrame()
for part_id,part in enumerate(feedback_group['ID'].unique().tolist()):
    file = feedback_group[feedback_group["ID"]== part]    
    ## indices of recorded feedback instances
    part_indices = file.index.tolist()
    # ## state before reordering weight feedback elements
    # app_ids = []
    # app_ids_idx = []
    ## state after reordering weight feedback elements
    app_ids_cl = []
    app_ids_idx_cl = []
    ## state before current reordering action
    prev_idx = None
    prev_app_id = None
    prev_value = None
    ##
    for idx in part_indices:
        app_id = file.loc[idx]['App ID']
        app_ids_cl.append(app_id)
        app_ids_idx_cl.append(idx)
        value = file.loc[idx].Value 
        ## GET new weight feedback element recorded 
        ## since previous recorded feedback instance
        ## store them in new_wf_els
        if len(app_ids_cl)>1:
            new_wf_els = copy.deepcopy(value)
            # if app_id not in app_ids:# remove previous weight feedback from current
            for el in prev_value:
                if el in new_wf_els:
                    new_wf_els.remove(el)
            # else:
            #     ## remove previous weight feedback from current
            #     for el in prev_value:
            #         if el in cl_value:
            #             cl_value.remove(el)
            #     ## add the weights from last feedback on same app
            #     file.loc[app_ids_idx[len(app_ids) - 1 - app_ids[::-1].index(app_id)]].Value
        else:
            new_wf_els = copy.deepcopy(value)
        ## CASE 1: new_wf_els is not empty
        ## Disrtibute each new weight feedback element 
        ## to the correct feedback instance
        cl_cur_value = []
        cl_prev_value = []
        cl_missed_rec_values = {}
        cl_missed_rec_attributes = {}
        # missed_weight_only_fd_inst = {}
        for el in new_wf_els:
            underscore_idx = el['attribute'].index('_')        
            fd_app_id =  int(el['attribute'][0:underscore_idx])
            el['attribute'] = el['attribute'][underscore_idx+1:]
            ## CASE 1.1: include it to current feedback instance (idx)
            if fd_app_id == app_id:     
                ## If there is previous label feedback for current app_id 
                ## copy weights from there - 
                ## participants saw last set weights for each app
                if len(cl_cur_value)==0 and app_ids_cl.count(fd_app_id) > 1: 
                    matched_idx = app_ids_idx_cl[len(app_ids_cl) - 1 - (app_ids_cl[0:len(app_ids_cl)-1][::-1].index(fd_app_id)+1)]
                    cl_cur_value = copy.deepcopy(file.loc[matched_idx].Value)
                    # print("p_id",file.loc[idx]['ID'],"idx",idx,"app_id",app_id,
                    #       "prev app_id idx",app_ids_idx[len(app_ids) - 1 - (app_ids[0:len(app_ids)-1][::-1].index(fd_app_id)+1)])
                    # print( cl_cur_value)
                    # print(cl_value)
                cl_cur_value = update_clear_value(cl_cur_value, el)
                # ## If attribute already in cl_cur_value
                # ## update value
                # ## else add element to cl_cur_value
                # if len(cl_cur_value):                    
                #     attr_found = False
                #     for el_prev_i,el_prev in enumerate(cl_cur_value):
                #         if el_prev['attribute'] == el['attribute']:# case not found in recorded data
                #             # print("p_id",file.loc[idx]['ID'],"idx",idx,"new weight for already set feature weight: prev_value",cl_cur_value[el_prev_i],"curr_value",el)
                #             cl_cur_value[el_prev_i]['value'] = el['value']
                #             attr_found = True
                #             break
                #     if not attr_found:
                #         cl_cur_value.append(el)
                #         # print("p_id",file.loc[idx]['ID'],"idx",idx,"new weight for not set feature weight",el)
                # else:
                #     cl_cur_value.append(el)
            ## CASE 1.2: include it to previous feedback instance (prev_idx)
            ## cases 1.2 and 1.3 fix recording order of weight feedback elements
            elif fd_app_id == prev_app_id:# append to previous feedback instance
                ## Get previous clean value
                if len(cl_prev_value)==0:                    
                    cl_prev_value = copy.deepcopy(file.loc[prev_idx].Value)
                cl_prev_value = update_clear_value(cl_prev_value, el)
                # ## If attribute already in cl_prev_value
                # ## update value
                # ## else add element to cl_prev_value
                # if len(cl_prev_value):                    
                #     attr_found = False
                #     for el_prev_i,el_prev in enumerate(cl_prev_value):
                #         if el_prev['attribute'] == el['attribute']:# case not found in recorded data
                #             # print("p_id",file.loc[idx]['ID'],"idx",idx,"new weight for already set feature weight: prev_value",cl_cur_value[el_prev_i],"curr_value",el)
                #             cl_prev_value[el_prev_i]['value'] = el['value']
                #             attr_found = True
                #             break
                #     if not attr_found:
                #         cl_prev_value.append(el)
                #         # print("p_id",file.loc[idx]['ID'],"idx",idx,"new weight for not set feature weight",el)
                # else:
                #     cl_prev_value.append(el)
                ###############TOBEDELETED###########
                # ## check if attribute already in prev_value
                # attr_found = False
                # for el_prev_i,el_prev in enumerate(prev_value):
                #     if el_prev['attribute'] == el['attribute']:
                #         # print("p_id",file.loc[idx]['ID'],"idx",idx,"prev_value",prev_value,"curr_value",el)
                #         prev_value[el_prev_i]['value'] = el['value']
                #         file.at[prev_idx,'Value'] = prev_value
                #         attr_found = True
                #         break
                # if not attr_found:
                #     file.at[prev_idx,'Value'] = file.loc[prev_idx].Value+[el]
                #     # print("p_id",file.loc[idx]['ID'],"idx",idx,"missed rec",el)
            ## CASE 1.3: include it to previous feedback instance (prev_idx)
            else:
                if fd_app_id not in cl_missed_rec_values:
                    cl_missed_rec_values[fd_app_id] = []
                    cl_missed_rec_attributes[fd_app_id] = None
                ## If there is previous label feedback for current app_id 
                ## copy weights from there - 
                ## participants saw last set weights for each app
                if len(cl_missed_rec_values[fd_app_id])==0 and fd_app_id in app_ids_cl: 
                    matched_idx = app_ids_idx_cl[len(app_ids_cl) - 1 - (app_ids_cl[0:len(app_ids_cl)-1][::-1].index(fd_app_id)+1)]
                    cl_missed_rec_values[fd_app_id] = copy.deepcopy(file.loc[matched_idx].Value)
                    cl_missed_rec_attributes[fd_app_id] = file.loc[matched_idx].Attribute
                cl_missed_rec_values[fd_app_id] = update_clear_value(cl_missed_rec_values[fd_app_id], el)
                ########checks########################
                # if fd_app_id in app_ids:
                #     print("p_id",file.loc[idx]['ID'],"idx",idx,"fd_app_id in previous app_ids",fd_app_id)
                # else:
                #     print("p_id",file.loc[idx]['ID'],"idx",idx,"fd_app_id not in previous app_ids",fd_app_id)
        ## handle missed feedback records
        if len(cl_missed_rec_values):
            for missed_app_id in cl_missed_rec_values:
                new_row_dict = {"serial number": None, 
                                "timestamp": None,
                                "ID":part,
                                "App ID":missed_app_id,
                                "Function":"select_reject_application",
                                "Attribute":cl_missed_rec_attributes[missed_app_id],
                                "Value":None,
                                "Lower bound":0.0,"Upper bound":0.0}
                new_row = pd.DataFrame(new_row_dict,index=[count_idx])
                new_row.at[count_idx,'Value'] = cl_missed_rec_values[missed_app_id]
                # print(count_idx, missed_app_id,cl_missed_rec_attributes[missed_app_id])
                ########TOBEDELETED#############
                # if missed_app_id in app_ids:
                #     missed_matched_rec = copy.deepcopy(file.loc[app_ids_idx[len(app_ids) - 1 - app_ids[::-1].index(missed_app_id)]])
                #     new_row.at[count_idx,"Attribute"] = missed_matched_rec["Attribute"]
                #     new_row.at[count_idx,"Value"] = missed_matched_rec["Value"]
                #     # if part == '5d66f23706e589000157e7ec':
                #     #     print("p_id",file.loc[idx]['ID'],"idx",idx,"data",missed_weight_only_fd_inst)
                #     #     print(new_row)
                #     if len(new_row.loc[count_idx]["Value"]):
                #         for el in missed_weight_only_fd_inst[missed_app_id]:
                #             attr_found = False
                #             for el2_i,el2 in enumerate(new_row.loc[count_idx]["Value"]):
                #                 if el2['attribute'] == el['attribute']:
                #                     val = copy.deepcopy(new_row.loc[count_idx]["Value"])
                #                     val[el2_i]['value'] = el['value']
                #                     new_row.at[count_idx,"Value"] = val
                #                     attr_found = True
                #                     break 
                #             if not attr_found:
                #                 val = copy.deepcopy(new_row.loc[count_idx]["Value"])
                #                 val.append(el)
                #                 new_row.at[count_idx,"Value"] = val
                #     else:
                #         new_row.at[count_idx,"Value"] = missed_weight_only_fd_inst[missed_app_id]
                # else:
                #     new_row.at[count_idx,"Value"] = missed_weight_only_fd_inst[missed_app_id]
                
                split_idx = file.index.tolist().index(idx)
                file = pd.concat([file.iloc[:split_idx], new_row, file.iloc[split_idx:]])
                app_ids_cl.insert(len(app_ids_cl)-1, missed_app_id)
                app_ids_idx_cl.insert(len(app_ids_idx_cl)-1, count_idx)
                count_idx = count_idx - 1                

            ########checks########################
            # for missed_app_id in missed_weight_only_fd_inst:
            #     print("p_id",file.loc[idx]['ID'],"idx",idx,"data",Counter([missed_el['attribute'] for missed_el in missed_weight_only_fd_inst[missed_app_id]]))

            # print("p_id",file.loc[idx]['ID'],"idx",idx,"data",missed_weight_only_fd_inst)
            # if len(missed_weight_only_fd_inst)>1:
            #     for missed_app_id in missed_weight_only_fd_inst:
            #         if missed_app_id in app_ids:
            #             print("p_id",file.loc[idx]['ID'],"idx",idx,"exists:",missed_app_id)
            #         else:
            #             print("p_id",file.loc[idx]['ID'],"idx",idx,"does not exist:",missed_app_id)
        ## Update data of previous feedback record
        if len(cl_prev_value):
            file.at[prev_idx,'Value'] = cl_prev_value
        ## Skip last feedback instance
        ## because we are not sure if there are missed feedback weight elements
        if idx==part_indices[-1]:
            file.drop(index = idx,inplace = True)
        else:
            ## CASE 2: new_wf_els was empty
            ## check if app_id appeared previously
            ## and copy weights feedback
            if len(cl_cur_value)==0 and app_ids_cl.count(app_id) > 1: 
                matched_idx = app_ids_idx_cl[len(app_ids_cl) - 1 - (app_ids_cl[0:len(app_ids_cl)-1][::-1].index(app_id)+1)]
                cl_cur_value = copy.deepcopy(file.loc[matched_idx].Value)            
            file.at[idx,'Value'] = cl_cur_value                  
        ## Update state variables
        prev_idx = idx
        prev_app_id = app_id
        prev_value = copy.deepcopy(value)
    df_all = pd.concat([df_all,file])
df_all

fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100067] {100038: []}
matched idx 1395 data {100038: []}
fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100067] {100038: [{'attribute': 'Mobile reachable', 'value': '0.015'}]}
fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100067] {100038: [{'attribute': 'Mobile reachable', 'value': '0.015'}, {'attribute': 'Income', 'value': '0.028'}]}
fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100067] {100038: [{'attribute': 'Mobile reachable', 'value': '0.015'}, {'attribute': 'Income', 'value': '0.028'}, {'attribute': 'Number of children', 'value': '0.02'}]}
fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100067] {100038: [{'attribute': 'Mobile reachable', 'value': '0.015'}, {'attribute': 'Income', 'value': '0.028'}, {'attribute': 'Number of children', 'value': '0.02'}, {'attribute': 'Owns Property', 'value': '0.008'}]}
fd_app_id 100038 app_ids_cl [100001, 104458, 100038, 100028, 100

,serial number,timestamp,ID,App ID,Function,Attribute,Value,Lower bound,Upper bound
3,8,2022-12-20 14:39:40.335007,5e909231933994023e8b3c4d,106823,select_reject_application,fair,[],0.0,0.0
5,13,2022-12-20 14:39:53.711255,5e909231933994023e8b3c4d,106647,select_reject_application,unfair,[],0.0,0.0
7,18,2022-12-20 14:40:32.725726,5e909231933994023e8b3c4d,106401,select_reject_application,unfair,[],0.0,0.0
9,22,2022-12-20 14:41:09.721315,5e909231933994023e8b3c4d,106861,select_reject_application,unfair,[],0.0,0.0
11,27,2022-12-20 14:41:23.548989,5e909231933994023e8b3c4d,100001,select_reject_application,fair,[],0.0,0.0
...,...,...,...,...,...,...,...,...,...
3992,94,2022-12-20 16:50:41.065143,5d66f23706e589000157e7ec,100241,select_reject_application,fair,[{'attribute': 'Number of times social circle ...,0.0,0.0
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,[{'attribute': 'Number of times social circle ...,0.0,0.0
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,[{'attribute': 'Number of times social circle ...,0.0,0.0
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"[{'attribute': 'Gender', 'value': '0'}, {'attr...",0.0,0.0


In [17]:
feedback_group = feedback_df[feedback_df["Function"] == "select_reject_application"]
feedback_group

,serial number,timestamp,ID,App ID,Function,Attribute,Value,Lower bound,Upper bound
3,8,2022-12-20 14:39:40.335007,5e909231933994023e8b3c4d,106823,select_reject_application,fair,[],0.0,0.0
5,13,2022-12-20 14:39:53.711255,5e909231933994023e8b3c4d,106647,select_reject_application,unfair,[],0.0,0.0
7,18,2022-12-20 14:40:32.725726,5e909231933994023e8b3c4d,106401,select_reject_application,unfair,[],0.0,0.0
9,22,2022-12-20 14:41:09.721315,5e909231933994023e8b3c4d,106861,select_reject_application,unfair,[],0.0,0.0
11,27,2022-12-20 14:41:23.548989,5e909231933994023e8b3c4d,100001,select_reject_application,fair,[],0.0,0.0
...,...,...,...,...,...,...,...,...,...
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"[{'attribute': '106488_Gender', 'value': '0'},...",0.0,0.0
4000,165,2022-12-20 16:55:55.457697,5d66f23706e589000157e7ec,103570,select_reject_application,unfair,"[{'attribute': '103570_Goods Price', 'value': ...",0.0,0.0


In [71]:
pid = '5d66a40c141c9800016a08a2'
pid = '6120211d8e1eab16fcb7ad69'
# p_id = '5e465ab4c07877130185c306'## case 3: 3 consecutive times same app
# pid = "5ebed23204aa470f9e1299bb"##TODO case 1
pid = '5f02fbee212873485f6ce5b8'##TODO general debug
pid = '5f256074297aac1d8ef7382d'## TODO general debug
# pid = '60dc5726c777d9a0e550d1ca'## case 1 idx 497-498
# pid = '60e3303c9f5ea2fc726d58d8'## case 1 -6 -7 you need to update apps_ids_idx when new row is added with previous label feedback
fd_p = feedback_group[feedback_group.ID==pid]
fd_p

,serial number,timestamp,ID,App ID,Function,Attribute,Value,Lower bound,Upper bound
741,18,2022-12-20 14:41:24.422959,5f256074297aac1d8ef7382d,100001,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
743,26,2022-12-20 14:42:14.476614,5f256074297aac1d8ef7382d,100005,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
745,34,2022-12-20 14:42:36.940511,5f256074297aac1d8ef7382d,100013,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
747,41,2022-12-20 14:43:01.965217,5f256074297aac1d8ef7382d,100028,select_reject_application,unfair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
762,67,2022-12-20 14:44:08.393324,5f256074297aac1d8ef7382d,104458,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
764,74,2022-12-20 14:44:30.525717,5f256074297aac1d8ef7382d,100168,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
766,81,2022-12-20 14:44:49.316485,5f256074297aac1d8ef7382d,105239,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
770,94,2022-12-20 14:45:29.711164,5f256074297aac1d8ef7382d,102996,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
772,109,2022-12-20 14:46:05.468824,5f256074297aac1d8ef7382d,105370,select_reject_application,unfair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0
774,116,2022-12-20 14:46:35.798049,5f256074297aac1d8ef7382d,105399,select_reject_application,fair,"[{'attribute': '106647_Owns Property', 'value'...",0.0,0.0


In [100]:
len(fd_p['App ID'].unique().tolist()) == len(fd_p)

True

In [102]:
# fd_p.index.tolist()

In [68]:
fd_p.loc[497].Value

[{'attribute': '100445_Owns Car', 'value': '0.073'}]

In [72]:
df_all_p = df_all[df_all.ID==pid]
df_all_p

,serial number,timestamp,ID,App ID,Function,Attribute,Value,Lower bound,Upper bound
-3,None,None,5f256074297aac1d8ef7382d,106647,select_reject_application,None,"[{'attribute': 'Owns Property', 'value': '0.01...",0.0,0.0
741,18,2022-12-20 14:41:24.422959,5f256074297aac1d8ef7382d,100001,select_reject_application,fair,"[{'attribute': 'Years in current employment', ...",0.0,0.0
743,26,2022-12-20 14:42:14.476614,5f256074297aac1d8ef7382d,100005,select_reject_application,fair,"[{'attribute': 'Income', 'value': '0.057'}, {'...",0.0,0.0
745,34,2022-12-20 14:42:36.940511,5f256074297aac1d8ef7382d,100013,select_reject_application,fair,"[{'attribute': 'Owns Property', 'value': '0.04...",0.0,0.0
747,41,2022-12-20 14:43:01.965217,5f256074297aac1d8ef7382d,100028,select_reject_application,unfair,"[{'attribute': 'Goods Price', 'value': '0.06'}...",0.0,0.0
762,67,2022-12-20 14:44:08.393324,5f256074297aac1d8ef7382d,104458,select_reject_application,fair,"[{'attribute': 'Income', 'value': '0.069'}]",0.0,0.0
764,74,2022-12-20 14:44:30.525717,5f256074297aac1d8ef7382d,100168,select_reject_application,fair,"[{'attribute': 'Income', 'value': '0.054'}]",0.0,0.0
766,81,2022-12-20 14:44:49.316485,5f256074297aac1d8ef7382d,105239,select_reject_application,fair,"[{'attribute': 'Income', 'value': '0.054'}]",0.0,0.0
770,94,2022-12-20 14:45:29.711164,5f256074297aac1d8ef7382d,102996,select_reject_application,fair,"[{'attribute': 'Highest education level', 'val...",0.0,0.0
772,109,2022-12-20 14:46:05.468824,5f256074297aac1d8ef7382d,105370,select_reject_application,unfair,"[{'attribute': 'Income', 'value': '0.067'}, {'...",0.0,0.0


In [66]:
df_all_p = df_all[df_all.ID==pid]
df_all_p.loc[497].Value

[{'attribute': 'Owns Car', 'value': '0.073'}]

## How many times participants suggested a weight change for each attribute

In [4]:
feedback_group_weight_dict = feedback_df[feedback_df["Function"] == "select_reject_application"]

## convert string values to list of dicts
for i in feedback_group_weight_dict.index :
    s = feedback_group_weight_dict['Value'].loc[i]
    # print(i,feedback_df['App ID'].loc[i])
    s = s.replace("'", "\"")
    s = s.replace("Region\"s Rating", "Region\'s Rating")
    s = s.replace("Region\"s Normalized Population", "Region\'s Normalized Population")
    feedback_group_weight_dict.loc[[i], "Value"] = pd.Series([json.loads(s)], index=[i])
 
## keep only feature weights relevant to each APP ID
all_new_values = []
for i in feedback_group_weight_dict.index :
    app_id = feedback_group_weight_dict['App ID'].loc[i]
    value = feedback_group_weight_dict['Value'].loc[i]
    new_value = {}
    if value:
        for v_dict in value:
            _idx = v_dict['attribute'].index('_')
            if app_id == np.int64(v_dict['attribute'][0:_idx]):
                attr = v_dict['attribute'][(_idx+1):]
                new_value[attr]=np.float64(v_dict['value'])
    all_new_values.append(new_value)    
feedback_group_weight_dict['Value'] = all_new_values
feedback_group_weight_dict

,serial number,timestamp,ID,App ID,Function,Attribute,Value,Lower bound,Upper bound
3,8,2022-12-20 14:39:40.335007,5e909231933994023e8b3c4d,106823,select_reject_application,fair,{},0.0,0.0
5,13,2022-12-20 14:39:53.711255,5e909231933994023e8b3c4d,106647,select_reject_application,unfair,{},0.0,0.0
7,18,2022-12-20 14:40:32.725726,5e909231933994023e8b3c4d,106401,select_reject_application,unfair,{},0.0,0.0
9,22,2022-12-20 14:41:09.721315,5e909231933994023e8b3c4d,106861,select_reject_application,unfair,{},0.0,0.0
11,27,2022-12-20 14:41:23.548989,5e909231933994023e8b3c4d,100001,select_reject_application,fair,{},0.0,0.0
...,...,...,...,...,...,...,...,...,...
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,{'Number of times social circle defaulted on l...,0.0,0.0
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,{'Number of times social circle at default ris...,0.0,0.0
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"{'Gender': 0.0, 'Income': 0.054, 'Application ...",0.0,0.0
4000,165,2022-12-20 16:55:55.457697,5d66f23706e589000157e7ec,103570,select_reject_application,unfair,"{'Goods Price': 0.034, 'Gender': 0.0, 'Loan Cr...",0.0,0.0


In [19]:
labels_with_weights_all = feedback_group_weight_dict[[True if len(val) else False for val in feedback_group_weight_dict["Value"]]]

total_clickcs_per_attr ={}
number_part_per_attr = {}
for idx in labels_with_weights_all.index:
    ch_weights = labels_with_weights_all.loc[idx]['Value']
    for attr in ch_weights:
        if attr not in total_clickcs_per_attr:
            total_clickcs_per_attr[attr] = 0
        if attr not in number_part_per_attr:
            number_part_per_attr[attr] = []
        if labels_with_weights_all.loc[idx]['ID'] not in number_part_per_attr[attr]:
            number_part_per_attr[attr].append(labels_with_weights_all.loc[idx]['ID'])
        total_clickcs_per_attr[attr] = total_clickcs_per_attr[attr] + 1
for attr in number_part_per_attr:
    number_part_per_attr[attr] = len(number_part_per_attr[attr])

data = []
for attr in total_clickcs_per_attr:
    data.append({'attribute':attr, 
                 'Total_Weight_Changes':total_clickcs_per_attr[attr],
                 'Number of participants clicked':number_part_per_attr[attr]})

weight_changes_df = pd.DataFrame(data)
weight_changes_df.sort_values(by='Number of participants clicked', ascending=False, inplace = True)
weight_changes_df.fillna("NA", inplace = True)
weight_changes_df.reset_index(drop = True, inplace = True)
weight_changes_df
# weight_changes_df.to_csv("How many times participants suggested a weight change for each attribute.csv")

,attribute,Total_Weight_Changes,Number of participants clicked
0,Owns Car,96,34
1,Gender,109,23
2,Years in current employment,52,23
3,Income type,58,22
4,Housing situation,35,20
5,Highest education level,39,18
6,Income,90,18
7,Occupation Type,45,18
8,Age,48,17
9,Family status,28,17


## Copied

In [ ]:
## load all participants' interaction data with the UI
feedback_df = pd.read_csv(folder+'data/processed_data/Feedback_final.csv', delimiter=',', index_col=0)
feedback_df.drop(columns = ["Lower bound","Upper bound"],inplace = True) # blank columns
## get only feedback-related interactions with the UI
feedback_df = feedback_df[feedback_df["Function"] == "select_reject_application"] 
## load application data shown to participants through the UI
applications_df = pd.read_csv("Applications_Converted.csv", index_col=0)
## get list of weights before feedback integration in right order
features_in_trainingOrder = ['Number of children', 'Income', 'Loan Credit amount ', 'Loan annuity',
       'Goods Price', "Region's Normalized Population", 'Age',
       'Years in current employment', 'Years since changing registration', 'Years since changing  identity document',
       'Has Mobile', 'Has Employee Phone', 'Has Work Phone', 'Mobile reachable',
       'Has Phone', 'Has email', 'Number of family members', "Region's Rating",
       'Region & City Rating', 'Application Hour',
       'Contact address located in Registration region', 'Work address located in Registration region',
       'Contact address located in work region', 'Contact city in Registration city',
       'Work address in Registration city', 'Contact city in work city',
       'Number of times social circle at default risk for 30 days', 'Number of times social circle defaulted on loan for 30 days',
       'Number of times social circle were at risk to default on loan for 60 days', 'Number of times social circle defaulted on loan for 60 days',
       'Years since changing phone', 'Number of Credit Bureau enquiries 1 hour before application',
       'Number of Credit Bureau enquiries 1 day before application', 'Number of Credit Bureau enquiries 1 week before application',
       'Number of Credit Bureau enquiries 1 month before application', 'Number of Credit Bureau enquiries 3 month before application',
       'Number of Credit Bureau enquiries 1 year before application', 'Installments', 'Gender',
       'Owns Car', 'Owns Property', 'Accompanied while applying',
       'Income type', 'Highest education level',
       'Family status', 'Housing situation', 'Occupation Type',
       'Application Day', 'Employer organization']
attributes = {'Gender':'CODE_GENDER',
              'Family status':'NAME_FAMILY_STATUS',
              'Age':'AGE',
              "Region's Rating":'REGION_RATING_CLIENT',
              'Region & City Rating':'REGION_RATING_CLIENT_W_CITY',
              'Number of times social circle at default risk for 30 days':'OBS_30_CNT_SOCIAL_CIRCLE',
             'Number of times social circle defaulted on loan for 30 days':'DEF_30_CNT_SOCIAL_CIRCLE',
             'Number of times social circle were at risk to default on loan for 60 days':'OBS_60_CNT_SOCIAL_CIRCLE',
             'Number of times social circle defaulted on loan for 60 days':'DEF_60_CNT_SOCIAL_CIRCLE',
             'Number of children':'CNT_CHILDREN', 
              'Income':'AMT_INCOME_TOTAL', 
              'Loan Credit amount ':'AMT_CREDIT', 
              'Loan annuity':'AMT_ANNUITY',
               'Goods Price':'AMT_GOODS_PRICE', 
              "Region's Normalized Population":'REGION_POPULATION_RELATIVE',
               'Years in current employment':'YEARS_EMPLOYED', 
              'Years since changing registration':'YEARS_REGISTRATION', 
              'Years since changing  identity document':'YEARS_ID_PUBLISH',
               'Has Mobile':'FLAG_MOBIL', 
              'Has Employee Phone':'FLAG_EMP_PHONE', 
              'Has Work Phone':'FLAG_WORK_PHONE', 
              'Mobile reachable':'FLAG_CONT_MOBILE',
               'Has Phone':'FLAG_PHONE', 
              'Has email':'FLAG_EMAIL', 
              'Number of family members':'CNT_FAM_MEMBERS', 
              'Application Hour':'HOUR_APPR_PROCESS_START',
       'Contact address located in Registration region':'REG_REGION_NOT_LIVE_REGION', 
              'Work address located in Registration region':'REG_REGION_NOT_WORK_REGION',
       'Contact address located in work region':'LIVE_REGION_NOT_WORK_REGION', 
              'Contact city in Registration city':'REG_CITY_NOT_LIVE_CITY',
       'Work address in Registration city':'REG_CITY_NOT_WORK_CITY', 
              'Contact city in work city':'LIVE_CITY_NOT_WORK_CITY',
       'Years since changing phone':'YEARS_LAST_PHONE_CHANGE', 
              'Number of Credit Bureau enquiries 1 hour before application':'AMT_REQ_CREDIT_BUREAU_HOUR',
       'Number of Credit Bureau enquiries 1 day before application':'AMT_REQ_CREDIT_BUREAU_DAY', 
              'Number of Credit Bureau enquiries 1 week before application':'AMT_REQ_CREDIT_BUREAU_WEEK',
       'Number of Credit Bureau enquiries 1 month before application':'AMT_REQ_CREDIT_BUREAU_MON', 
              'Number of Credit Bureau enquiries 3 month before application':'AMT_REQ_CREDIT_BUREAU_QRT',
       'Number of Credit Bureau enquiries 1 year before application':'AMT_REQ_CREDIT_BUREAU_YEAR', 
              'Installments':'NAME_CONTRACT_TYPE', 
       'Owns Car':'FLAG_OWN_CAR', 
              'Owns Property':'FLAG_OWN_REALTY', 
              'Accompanied while applying':'NAME_TYPE_SUITE',
       'Income type':'NAME_INCOME_TYPE', 
              'Highest education level':'NAME_EDUCATION_TYPE',
              'Housing situation':'NAME_HOUSING_TYPE', 
              'Occupation Type':'OCCUPATION_TYPE ',
       'Application Day':'WEEKDAY_APPR_PROCESS_START', 
              'Employer organization':'ORGANIZATION_TYPE'}
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
feature_weights = []
for feat in features_in_trainingOrder:
    feature_weights.append(feature_weights_dict[feat])



In [ ]:
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
    
av_weight_ch_dict = {}
for attr in attributes:
    av_weight_ch_dict[attributes[attr]] = []
for i in feedback_df.index:
    app_id = feedback_df['App ID'].loc[i]
    value_dict = feedback_df['Value'].loc[i]
    if value_dict:
        for attr in value_dict:
            av_weight_ch_dict[attributes[attr]].append(value_dict[attr] - feature_weights_dict[attr])
for attr in av_weight_ch_dict:
    av_weight_ch_dict[attr] = sum(av_weight_ch_dict[attr])/len(av_weight_ch_dict[attr])
    idx = list(attributes.values()).index(attr)
    av_weight_ch_dict[attr] = (av_weight_ch_dict[attr]/abs(feature_weights[idx]))*100
av_weight_ch_dict = sorted(av_weight_ch_dict.items(), key=lambda x:x[1], reverse=True)
av_weight_ch_dict

# Average weight change suggested by participants for each attribute

In [ ]:
import ast
attr_weight_change_dict ={}
for idx in labels_with_weights_all.index:
    ch_weights = ast.literal_eval(labels_with_weights_all.loc[idx]['Value'])['changed_weights']
    init_weights = ast.literal_eval(ast.literal_eval(labels_with_weights_all.loc[idx]['Value'])['initial_weights'])
    for v in ch_weights:
        ## find initial value of v['attribute']
        init_v = None
        for v2 in init_weights:
            if v2['attribute'] == v['attribute']:
                init_v = float(v2['value'])
                break
        ## take perc change of weight for v['attribute']
        change = (float(v['value']) - init_v)/init_v if init_v else np.inf
        ##
        if v['attribute'] not in attr_weight_change_dict:
            attr_weight_change_dict[v['attribute']] = []
        attr_weight_change_dict[v['attribute']].append(change)
for attr in attr_weight_change_dict:
    attr_weight_change_dict[attr] = sum(attr_weight_change_dict[attr])/len(attr_weight_change_dict[attr])
    attr_weight_change_dict[attr] = attr_weight_change_dict[attr]*100

data = []
for attr in attr_weight_change_dict:
    data.append({'Attribute':attr, 
                 'No. Part.':weight_changes_df[weight_changes_df['attribute']==attr]['Number of participants clicked'].values.tolist()[0],
                 'Av. Weight. Ch. %':attr_weight_change_dict[attr]})

weight_prc_changes_df = pd.DataFrame(data)
weight_prc_changes_df.sort_values(by='No. Part.', ascending=False, inplace = True)
weight_prc_changes_df.fillna("NA", inplace = True)
weight_prc_changes_df.reset_index(drop = True, inplace = True)
weight_prc_changes_df
# weight_prc_changes_df.to_csv("Average weight change suggested by participants for each attribute.csv")

## How  many times an application was rated by participants w.r.t each participant

In [ ]:
parti_app_rating_group= feedback_df.groupby(by = ["ID","App ID","Attribute"])

parti_app_rating_count = parti_app_rating_group.size().unstack(fill_value=0).stack()

data = []

app_id_list = feedback_df["App ID"].unique().tolist()
app_id_list.remove(0)

for i in range(0,len(parti_app_rating_count.index)):
    if (parti_app_rating_count.index[i][1] in app_id_list) and (parti_app_rating_count.index[i][2] in feedback_df[feedback_df["Function"] == "OKBUTTON_CLICKED_DECIDE_MODAL_Applications_List"]["Attribute"].unique().tolist()):

        data.append(
        {
            "Participant" : parti_app_rating_count.index[i][0],
            "Application_Label" : (parti_app_rating_count.index[i][1],parti_app_rating_count.index[i][2]) ,
            "Application" : parti_app_rating_count.index[i][1],
            "Label" : parti_app_rating_count.index[i][2],
            "Count": parti_app_rating_count.values[i]
        }
        )
    
parti_app_rating_df = pd.DataFrame(data)
parti_app_rating_df.head(5)

In [ ]:
data = []

for app_label in parti_app_rating_df["Application_Label"].unique():
    
    data.append(
    {
        "App_Label" : app_label,
        "Total_Ratings" : parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.sum(),
        "Mean": round(parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.mean(),2),
        "Std": round(parti_app_rating_df[parti_app_rating_df["Application_Label"]== app_label].Count.std(),2),
        "Number of Participants Labeled": parti_app_rating_df[(parti_app_rating_df["Application_Label"]== app_label) & (parti_app_rating_df["Count"] != 0)].shape[0]}
    )
    
    
    
each_participant_app_rating_df = pd.DataFrame(data)

each_participant_app_rating_df.sort_values(by='Total_Ratings', ascending=False, inplace = True)
each_participant_app_rating_df.fillna(0, inplace = True)
each_participant_app_rating_df.reset_index(drop = True, inplace = True)

applications_list = []
label_list = []
for i in range(0,len(each_participant_app_rating_df)):
    applications_list.append(each_participant_app_rating_df["App_Label"][i][0])
    label_list.append(each_participant_app_rating_df["App_Label"][i][1])

each_participant_app_rating_df["Application"] = applications_list
each_participant_app_rating_df['Label'] = label_list
each_participant_app_rating_df = each_participant_app_rating_df.loc[:, ["Application","Label","Total_Ratings","Mean","Std","Number of Participants Labeled"]]

each_participant_app_rating_df.sort_values(by='Application', ascending=True, inplace = True)
each_participant_app_rating_df.reset_index(drop = True, inplace = True)
each_participant_app_rating_df
# for index in each_participant_app_rating_df.index: 
#     app_id = each_participant_app_rating_df.loc[index,"Application"]
#     each_participant_app_rating_df.loc[index,"Predicted_decision"] = df[app_id]["Predicted_decision"]
#     each_participant_app_rating_df.loc[index,"Prediction_Confidence_Accepted"] = df[app_id]["Prediction_Confidence_Accepted"]

# each_participant_app_rating_df
#each_participant_app_rating_df.to_csv("How  many times an application was rated by Users w.r.t each participant.csv")

In [ ]:
#each_participant_app_rating_df.to_csv("How  many times an application was rated by Users w.r.t each participant.csv")

## Unfairness ratio w.r.t each participant

In [ ]:
data = []
for participant in parti_rating_df["Participant"].unique():
    
    try: 
        unfair_count = int(parti_rating_df[(parti_rating_df["Participant"] == participant) & (parti_rating_df["Rating"] == "unfair")]["Count"].values)
    except: 
        unfair_count = 0
    try: 
        fair_count = int(parti_rating_df[(parti_rating_df["Participant"] == participant) & (parti_rating_df["Rating"] == "checked")]["Count"].values)
    except: 
        fair_count = 0
    try: 
        unfairness_ratio = round(unfair_count/(fair_count+unfair_count),2)
    except: 
        unfairness_ratio = "NA"
           
    data.append(
        {
            "Participant" : participant,
            "Unfair_Count" : unfair_count, 
            "Checked_Count": fair_count,
            "Unfairness_Ratio": unfairness_ratio
        }
        )
    
unfairness_ratio_df = pd.DataFrame(data)

unfairness_ratio_df.sort_values(by='Unfairness_Ratio', ascending=True, inplace = True)
unfairness_ratio_df.reset_index(drop = True, inplace = True)
unfairness_ratio_df
#unfairness_ratio_df.to_csv("Unfairness ratio w.r.t each participant.csv") 

## !!!Mean and std of the suggested weight change for a feature w.r.t the original weight 

In [ ]:
attribute_weight_df = feedback_df[[True if isinstance(val,str) and "'changed_weights': [{" in val else False for val in feedback_df["Value"]]]
# attribute_weight_df = feedback_df[(feedback_df["Function"] == "select_reject_application") & (feedback_df["Value"] != "[]")]
attribute_weight_df.reset_index(drop = True, inplace = True)

In [ ]:
# attribute_weight_df
# attribute_weight_df[attribute_weight_df["ID"] == '5f1c55c8ea92af4d99d03137'].iloc[-1,6]
attribute_weight_df

In [ ]:
import ast
data = []

# participants_list = attribute_weight_df[attribute_weight_df["ID"] == "5c48be0496d59b000183e68d"]

for participant in attribute_weight_df["ID"].unique():
    
    wt_string = attribute_weight_df[attribute_weight_df["ID"] == participant].iloc[-1,6]
    dict_wt_string = ast.literal_eval(wt_string)
    
    data.append(
        {
            "Participant" : participant,
            "Count" : attribute_weight_df[attribute_weight_df["ID"] == participant].shape[0],
            "Application_Attribute_Wt": dict_wt_string
        }
        )

weight_dict_df = pd.DataFrame(data)

# Append the users     
weight_dict_df

In [ ]:
weight_dict_df.loc[0, "Application_Attribute_Wt"]['changed_weights']
attribute_weight_df.loc[ind, "Value"]

In [ ]:
import ast
data = []
for p_id in attribute_weight_df['ID'].unique():
    for ind in attribute_weight_df[attribute_weight_df['ID']==p_id].index:
        feed = ast.literal_eval(attribute_weight_df.loc[ind, "Value"])['changed_weights']
        for j in range(len(feed)):
            current_dict_value_list = list(feed[j].values())    
            appplication_id = attribute_weight_df.loc[ind, "App ID"]
            attribute = current_dict_value_list[0]
            attribute_weight = current_dict_value_list[1]
    
            data.append(
                {
                    "Participant" : p_id,
                    "Application_ID" : appplication_id,
                    "Attribute"  :attribute,
                    "Attribute_Weight": attribute_weight
                }
            )

application_attribute_weight_df = pd.DataFrame(data)

application_attribute_weight_df["Application_ID"] = application_attribute_weight_df["Application_ID"].astype(int)
application_attribute_weight_df["Attribute_Weight"] = application_attribute_weight_df["Attribute_Weight"].astype(float)

# Add the remaining participants
labels_without_weights_dict = []

for i in labels_without_weights.unique().tolist():
    
    for attribute in application_attribute_weight_df["Attribute"].unique():
        labels_without_weights_dict.append({
        "Participant": i,
        "Application_ID": 0,
        "Attribute":attribute,
        "Attribute_Weight": 0})


application_attribute_weight_df = pd.concat([application_attribute_weight_df,pd.DataFrame(labels_without_weights_dict)],axis=0, ignore_index = True)
application_attribute_weight_df
# application_attribute_weight_df.to_csv("Application_Attribute_Weights.csv")

In [ ]:
participants_weight_group= application_attribute_weight_df.groupby(by = ["Participant","Attribute"])

attribute_sum = round(participants_weight_group["Attribute_Weight"].sum().unstack(fill_value=0).stack(),2)

attribute_count = round(participants_weight_group["Attribute_Weight"].count().unstack(fill_value=0).stack(),2)

data = []

for i in range(0,len(attribute_sum.index)):
    
    data.append(
    {
        "Participant" : attribute_sum.index[i][0],
        "Attribute" : attribute_sum.index[i][1],
        "Weight": attribute_sum.values[i],
        "Count" : attribute_count.values[i]
    }
    )
    
participant_weight_attribute_df = pd.DataFrame(data)
participant_weight_attribute_df

In [ ]:
data = []

for attribute in participant_weight_attribute_df["Attribute"].unique():
    
    data.append(
    {
        "Attribute" : attribute,
        "Number of Times Altered" : participant_weight_attribute_df[participant_weight_attribute_df["Attribute"]== attribute]["Count"].sum(),
        "Total_Weight" : round(participant_weight_attribute_df[participant_weight_attribute_df["Attribute"]== attribute].Weight.sum(),2),
        "Mean": round(participant_weight_attribute_df[participant_weight_attribute_df["Attribute"]== attribute].Weight.mean(),3),
        "Std": round(participant_weight_attribute_df[participant_weight_attribute_df["Attribute"]== attribute].Weight.std(),2),
        "Number of Participants Altered" : participant_weight_attribute_df[(participant_weight_attribute_df["Attribute"]== attribute) & (participant_weight_attribute_df["Count"] != 0)].shape[0]
    }
    )
    
    
each_participant_weight_attribute_df = pd.DataFrame(data)

each_participant_weight_attribute_df.sort_values(by='Number of Times Altered', ascending=False, inplace = True)
each_participant_weight_attribute_df.fillna("NA", inplace = True)
each_participant_weight_attribute_df.reset_index(drop = True, inplace = True)
each_participant_weight_attribute_df
# each_participant_weight_attribute_df.to_csv("Participants_Weight_Changing_Stats.csv")

In [ ]:
wt_wrt_old_weight_df = each_participant_weight_attribute_df.T
wt_wrt_old_weight_df.rename(columns=wt_wrt_old_weight_df.iloc[0],inplace =True)

for col in wt_wrt_old_weight_df.columns:
    wt_wrt_old_weight_df.loc["Old Weight",col] = ft_weight_dict[col]


wt_wrt_old_weight_df = wt_wrt_old_weight_df.T
wt_wrt_old_weight_df.reset_index(drop = True, inplace = True)

wt_wrt_old_weight_df["Old Weight"] = wt_wrt_old_weight_df["Old Weight"].astype(float)

for index in wt_wrt_old_weight_df.index:
    
    if wt_wrt_old_weight_df.loc[index,"Mean"] > wt_wrt_old_weight_df.loc[index,"Old Weight"]: 
        wt_wrt_old_weight_df.loc[index,"Increased/Reduced"] = "Increased"
    if wt_wrt_old_weight_df.loc[index,"Mean"] < wt_wrt_old_weight_df.loc[index,"Old Weight"]:
        wt_wrt_old_weight_df.loc[index,"Increased/Reduced"] = "Reduced"
     
    if wt_wrt_old_weight_df.loc[index,"Mean"] == wt_wrt_old_weight_df.loc[index,"Old Weight"]: 
        wt_wrt_old_weight_df.loc[index,"Increased/Reduced"] = "Same as Old"
        
wt_wrt_old_weight_df["Weight_Difference"] = wt_wrt_old_weight_df["Mean"] - wt_wrt_old_weight_df["Old Weight"] 

wt_wrt_old_weight_df
# wt_wrt_old_weight_df.to_csv("Mean and std of the suggested weight change for a feature w.r.t the original weight.csv")

In [ ]:
wt_wrt_old_weight_df["Increased/Reduced"].value_counts()

In [ ]:
wt_wrt_old_weight_df